* Part 1: Lognormal and exponential fit
* Part 2: Equilibrium + mining activity

In [23]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, ToggleButtons
from scipy.optimize import minimize


# Define the function to update the plot
def update_plot(sigma, mu, fit_type):
    
    # Define functions to calculate log-normal and exponential PDFs
    def lognormal_pdf(x, mu, sigma):
        return (1 / (x * sigma * np.sqrt(2 * np.pi))) * np.exp(- (np.log(x) - mu) ** 2 / (2 * sigma ** 2))

    def exponential_pdf(x, lambda_param):
        return lambda_param * np.exp(-lambda_param * x)

    # Function to minimize for least distance fit
    def least_distance_fit(lambda_param, x, lognormal_values):
        exponential_values = exponential_pdf(x, lambda_param)
        return np.sum((lognormal_values - exponential_values) ** 2)

    # Function to minimize for constant integral of x * PDF
    def constant_integral_fit(lambda_param, x, lognormal_values):
        exponential_values = exponential_pdf(x, lambda_param)
        return np.abs(np.sum(x * lognormal_values) - np.sum(x * exponential_values))

    
    x = np.linspace(0.01, 15, 1000)  # Avoid zero to prevent log issues

    # Log-normal PDF
    lognormal_values = lognormal_pdf(x, mu, sigma)

    # Exponential fit
    if fit_type == 'Total area':  # Total area fit
        sample = np.random.lognormal(mean=mu, sigma=sigma, size=100000)
        lambda_param = 1 / np.mean(sample)
    elif fit_type == 'Least distance':  # Least distance fit
        result = minimize(least_distance_fit, x0=1, args=(x, lognormal_values), bounds=[(0.01, 10)])
        lambda_param = result.x[0]
    else:  # Constant integral fit
        result = minimize(constant_integral_fit, x0=1, args=(x, lognormal_values), bounds=[(0.01, 10)])
        lambda_param = result.x[0]

    exponential_values = exponential_pdf(x, lambda_param)

    # Calculate areas of over-estimation and under-estimation
    under_estimation = lognormal_values - exponential_values

    # Create traces
    trace_lognormal = go.Scatter(x=x, y=lognormal_values, mode='lines', name='Log-normal PDF')
    trace_exponential = go.Scatter(x=x, y=exponential_values, mode='lines', name='Exponential PDF', line=dict(dash='dash'))

    # Plot positive and negative parts separately
    trace_under_positive = go.Scatter(x=x, y=-np.where(under_estimation < 0, 0, under_estimation), fill='tozeroy', mode='none', fillcolor='rgba(0, 255, 0, 0.5)', name='Under-estimation')
    trace_under_negative = go.Scatter(x=x, y=-np.where(under_estimation < 0, under_estimation, 0), fill='tozeroy', mode='none', fillcolor='rgba(255, 0, 0, 0.5)', name='Over-estimation')

    # Create figure
    fig = go.Figure(data=[trace_lognormal, trace_exponential, trace_under_positive, trace_under_negative])
    fig.update_layout(
        title=f'Log-normal (σ={sigma:.2f}, μ={mu:.2f}) vs Exponential Fit (λ={lambda_param:.2f}), Fit Type={fit_type})<br>'
              f'Log-normal Total Area: {np.trapz(lognormal_values, x):.2f}, '
              f'Exponential Total Area: {np.trapz(exponential_values, x):.2f}',
        xaxis_title='x',
        yaxis_title='Density',
        legend_title='Distributions'
    )

    fig.show()

# Create interactive widgets
sigma_slider = FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='σ:')
mu_slider = FloatSlider(min=0., max=2.0, step=0.1, value=0.0, description='μ:')
fit_toggle = ToggleButtons(options=[('Total Area', 'Total area'), ('Least Distance', 'Least distance'),('Total ressources','Total ressources')], description='Fit Type:')

# Use interact to update the plot with widgets
interact(update_plot, sigma=sigma_slider, mu=mu_slider, fit_type=fit_toggle)
print('')


interactive(children=(FloatSlider(value=0.5, description='σ:', max=2.0, min=0.1), FloatSlider(value=0.0, descr…

In [24]:
import numpy as np


def run_simulation(Params,nt,dt):
    Outvar = ['A',
            'Gamma',
            'C',
            'Y',
            'Rsoc',
            'time',
            'zeta',
            'omega',
            'r'
            ]

    Out = {k: np.zeros(nt) for k in Outvar}
    Out['Rsoc'][0]=Params['Rsoc']


    for i in range(nt):
        t = Out['time'][i]
        
        # Concentration
        Out['C'][i] = np.log(Params['Rtot']/Out['Rsoc'][i])/Params['Q']  
        
        # Progress and impact of improvement 
        Out['A'][i] = Params['A00']*np.exp(t*Params['beta_A0'])+Params['A0C']*np.exp(t*Params['beta_AC'])*Out['C'][i]
        Out['Gamma'][i] = Params['Gamma00']*np.exp(-t*Params['beta_gamma0'])+Params['Gamma0C']*np.exp(-t*Params['beta_gammaC'])/Out['C'][i]
        Out['zeta'][i] = Params['zeta0']*np.exp(t*Params['beta_zeta'])
        Out['r'][i] = 1-Out['Rsoc'][i]/Params['Rtot']
        
        # Production
        Out['Y'][i] = Out['A'][i]*np.exp(t*(Params['alpha']+Params['n']))
        
        # Wage share
        Out['omega'][i] = 1 - Out['Gamma'][i] - (Params['alpha'] + Params['n'] +Params['delta']) / Out['A'][i]
        
        if i!=nt-1:
            Out['Rsoc'][i+1] = Out['Rsoc'][i] + Out['zeta'][i]*Out['Y'][i]*dt
            Out['time'][i+1] = Out['time'][i] + dt
    return Out

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_figure(Out):

    # Define the variables to plot
    omega_values = Out['omega']
    relative_price = 1/Out['omega']
    r = Out['Rsoc']
    Y_values = Out['Y']
    Gamma_values = Out['Gamma']
    zeta_values = Out['zeta']
    A_values = Out['A']
    time_values = Out['time']
    Concentration = Out['C']

    # Create subplots: 4 rows, 2 columns
    fig = make_subplots(rows=4, 
                        cols=2, 
                        subplot_titles=("Wage share : omega", 
                                        "Relative price index",
                                        "Intermediate consumption : Gamma", 
                                        "Extraction intensity : Zeta", 
                                        "Raw efficiency: A", 
                                        "Share of resources extracted: r", 
                                        "Useful output: Y",
                                        "Resource concentration: C",
                                        "")
                        )

    # Add traces for each variable
    fig.add_trace(go.Scatter(x=time_values, y=omega_values, mode='lines', name='Omega'), row=1, col=1)
    fig.add_trace(go.Scatter(x=time_values, y=relative_price, mode='lines', name='Omega'), row=1, col=2)
    fig.add_trace(go.Scatter(x=time_values, y=Gamma_values, mode='lines', name='Gamma'), row=2, col=1)
    fig.add_trace(go.Scatter(x=time_values, y=zeta_values, mode='lines', name='Zeta'), row=2, col=2)
    fig.add_trace(go.Scatter(x=time_values, y=A_values, mode='lines', name='A'), row=3, col=1)
    fig.add_trace(go.Scatter(x=time_values, y=r, mode='lines', name='r'), row=3, col=2)
    fig.add_trace(go.Scatter(x=time_values, y=Y_values, mode='lines', name='Y'), row=4, col=1)
    fig.add_trace(go.Scatter(x=time_values, y=Concentration, mode='lines', name='C'), row=4, col=2)

    # Update layout for better visualization
    fig.update_layout(
        title="Finite resources extraction: impact on economy",
        legend_title="Variables",
        template="plotly_white"
    )

    # Show the figure
    #fig.show()
    return fig

In [25]:
Params = dict(
    # Production function parameter
    Y0 = 1.,            
    Gamma00 = 0.1,
    Gamma0C = 0.01,
    A00 = 0.,
    A0C = 1.,

    # Ressource parameter
    zeta0 = 0.,
    Rtot = 1.,
    Rsoc = 0.15,
    Q = 1.,

    # Society parameter
    delta = 0.03,
    alpha = 0.02,
    n = 0.02,

    # Improvement parameters
    beta_gamma0 = 0.1,
    beta_A0 = 0.1,
    beta_gammaC = 0.1,
    beta_AC = 0.1,
    beta_zeta = 0.1,
)

nt = 100
dt = 0.1

Out=run_simulation(Params,nt,dt)
fig=plot_figure(Out=Out)

In [29]:
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, ToggleButtons,VBox, interactive
from scipy.optimize import minimize
import numpy as np
import plotly.graph_objects as go
from IPython.display import display


# Interactive widgets for each parameter
widgets_list1 = []
widgets_list2 = []
for i,key in enumerate(Params.keys()):
    if i%2==0:
        widgets_list1.append(widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=Params[key], description=key))
    else:
        widgets_list2.append(widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=Params[key], description=key))

# Combine all sliders into a single VBox
widgets_box = widgets.HBox([widgets.VBox(widgets_list1),widgets.VBox(widgets_list2)])

# Define interactive function to update simulation and plot
def update_simulation(**kwargs):
    # Update Params dictionary with new values from sliders
    for key, value in kwargs.items():
        Params[key] = value
    
    # Rerun simulation with updated Params
    Out = run_simulation(Params, nt, dt)
    
    # Update existing plot with new simulation results
    update_plot(fig, Out)

# Function to update the plot with new simulation results
def update_plot(fig, Out):
    # Extract updated values from simulation results
    omega_values = Out['omega']
    r = Out['Rsoc']
    Y_values = Out['Y']
    Gamma_values = Out['Gamma']
    zeta_values = Out['zeta']
    A_values = Out['A']
    time_values = Out['time']
    Concentration = Out['C']

    # Update traces with new data
    fig.data[0].y = omega_values
    fig.data[1].y = Gamma_values
    fig.data[2].y = zeta_values
    fig.data[3].y = A_values
    fig.data[4].y = r
    fig.data[5].y = Y_values
    fig.data[6].y = Concentration

    # Update layout (if necessary)
    fig.update_layout(
        title="Finite resources extraction: impact on economy",
        legend_title="Variables",
        template="plotly_white"
    )

    # Show updated figure
    fig.show()

# Create interactive output with widgets
interactive_plot = interactive(update_simulation, **Params)
display(widgets_box, interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='Y0', max=3.0, min=-1.0), FloatSlider(value=0.1, desc…